# Softmax

实际上Softmax多配合于神经网络,在这里提出只是为了与LR相对应.

### 1 Softmax



softmax 可以用多分类问题,而不像LR等二类模型那样需要借助[One-vs-rest](https://zh.wikipedia.org/wiki/%E5%A4%9A%E5%85%83%E5%88%86%E7%B1%BB)推广到多分类.

设样本$x_i$的真实标签类别$y_i\in \{1,2,...,C\}$.同样我们需要借助One-Hot向量$y_i=(0,0,...,1,...0)$只有对应真实类别的那一维是1,长度为C.参照Naive Bayes(Application)文档中的例子.

**Ps:**

- 多分类: 是将示例归类为多个(大于两个)类别中的一类(将示例归为两类中的一类被称为二元分类).
- 多标签: 多标签分类要为每个示例预测多个标签,即一个示例可以同时被归为多个类别,比如一部电影可以有很多标签"爱情,动作,喜剧".

### 2 Softmax 后验估计

$z_c = w_cx+b$

$softmax=\sigma{(z_c)}=\frac{exp(z_c)}{\sum_{c=1}^{C}exp(z_c) }$

与LR类似,softmax输出的是样本$x_i$属于各个类别的后验概率估计值,对于给定数据集$T=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\},y_i\in\{0,1,...,C\}$

将每一个$y_i$转换成one-hot形式(对应类别的值为1,其余为0):

$Y=\begin{bmatrix}
1 &0  &...  &0 \\ 
 0&1  &...  &0 \\ 
... & ... & ... &... \\ 
 0&0  &...  & 1
\end{bmatrix}_{C,N}$




那么:

$P(Y=0|X)=(1,0,0...0)_{C}$

$P(Y=1|X)=(0,1,0...0)_{C}$

...

$P(Y=C|X)=(0,0,0...C)_{C}$

所以似然函数为:

$\prod P(Y=c|X)= \prod [\hat{y_i}]^{y_i}$

对数形式:

$logP(Y=c|X)=\sum_{i=1}^{N}y_ilog\hat{y_i}$

那么其损失函数(经验风险)为:

$loss(w,b)= -\frac{1}{N}\cdot \sum_{i=1}^{N}y_ilog\hat{y_i}$

**Ps:**

这里的似然函数也可以直接由LR的似然函数直接看出

$\prod_{i=1}^{N}[\hat{y_i}]^{y_i}\cdot[1-\hat{y_i}]^{1-y_i}$,相应的标签为1,其余的全为0,所以相应标签1带入似然函数,就得以得到上述形式

### 3 Parameters

假设现在我们的标签有4类,那么:

$a=\sigma(z)=\frac{exp(z_i)}{exp(z_1)+exp(z_2)+exp(z_3)+exp(z_4)}$

很明显,我们对其中一个$z_i$进行求导的话,会发现分母部分会有与$i$项无关的情况,所以我们将$a$对$z$求导分为两个部分:

(1) 当$i=j$:

$\frac{\partial a_i}{\partial z_i} = \frac{e^{z_i}\sum_j e^{z_j}-(e^{z_i})^2}{(\sum_j e^{z_j})^2} = \frac{e^{z_i}}{\sum_j e^{z_j}}-\frac{(e^{z_i})^2}{(\sum_j e^{z_j})^2} = a_i - a_i^2 = a_i(1-a_i)$分子可以求导，分母除了i项其他都为常数求导为0




(2) 当$i\neq j$:

$\frac{\partial a_i}{\partial z_j} =\frac{0\cdot \sum_j e^{z_j} - e^{z_j}e^{z_i}}{(\sum_j e^{z_j})^2} = -a_ia_j $

那么我们将两部分组合求损失函数对Z求导部分(链式法则):

$\frac{\partial L}{\partial z}= \frac{\partial L}{\partial a}\frac{\partial a}{\partial z}$ 

$= -\sum_j \frac{y_j}{a_j}\cdot \frac{\partial a_j}{\partial z_j}-\sum_i \frac{y_i}{a_i}\cdot \frac{\partial a_i}{\partial z_i}$

$=-\sum_i \frac{y_i}{a_i}a_i(1-a_i) + \sum_{i\neq j}\frac{y_j}{a_j}a_ia_j = -\sum_iy_i + \sum_iy_ia_i+a_i\sum_{i \neq j} y_j$

$ = -\sum_i y_i + a_i \sum y$ 


因为针对分类问题y的值只可能是一个1,其他都是0,所以结果为$\sum y = 1$

所以最终的结果是:$a_i - y_i$

<P style="color:orange">可以看出其和LR的结果是一样的.</p>

所以:

$dw_i=\frac{\partial loss}{\partial w_i}=x(a_i-y_i)$

$db_i=\frac{\partial loss}{\partial b_i}=(a_i-y_i)$

### 4 softmax Algorithm

下面给出softmax的算法.

**Softmax Algorithm:**

输入:样本数据$X=\{x_1,x_2,...,x_m\}$,样本标签$Y=\{y_1,y_2,...,y_m\}$,其中每个样本$x_i$有n个特征.每个标签$y_i\in \{1,2,...,C\}$,学习率$\alpha$

输出:样本预测标签$\hat{Y}$

(1) 将样本标签转换成"Hot"形式:$Y=\begin{bmatrix}
1 &0  &...  &0 \\ 
 0&1  &...  &0 \\ 
... & ... & ... &... \\ 
 0&0  &...  & 1
\end{bmatrix}_{C,N}$,即对应的类别为1,其余为0.初始化$W,b$

(2) 计算线性值$Z=WX+b$

(3) 计算预测值$A=softmax(Z)$

(4) 计算损失函数$loss(w,b)= -\frac{1}{m}\cdot \sum_{i=1}^{m}y_ilog\hat{y_i}$

(5) 更新参数:
    
$W{:=}W-\alpha dW$
    
$b:=W =\alpha db$

(6) 在损失精度阈值$\epsilon$内退出迭代,即$|loss^{k+1}-loss^{k}|<\epsilon$

### 5 Example of softmax

假设单个样本的线性回归计算完后的$Z$如下所示:

$Z = \begin{bmatrix}
5\\ 
2\\ 
-1\\
3\\ \end{bmatrix}$

其中每一列代表一个分类(上述有4个分类).那么softmax如下所示:

$t = \begin{bmatrix}
e^{5}\\ 
e^{2}\\ 
e^{-1}\\
e^{3}\\ \end{bmatrix} = \begin{bmatrix}
148.4\\ 
7.4\\ 
0.4\\
20.1\\ \end{bmatrix}$

$a = \frac{t}{176.3} = \begin{bmatrix}
0.842\\ 
0.042\\ 
0.002\\
0.114\\ \end{bmatrix}$

可以看出实际上4个分类的求和为1,也就是代表着,softmax实际上是将一个融合概率拆分成4个子概率,其中0.842最大,说明这个样本属于第一类的概率最大.